In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
# 1. Downgrade SymPy to fix the AttributeError
!pip install "sympy==1.12"

# 2. Re-install Unsloth (just to be safe)
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-qsdwcaas/unsloth_6f3ab81bbe254899946d4a18c793d982
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-qsdwcaas/unsloth_6f3ab81bbe254899946d4a18c793d982
  Resolved https://github.com/unslothai/unsloth.git to commit 6789c279d578278aca4af22f4ca31fc42829c9a4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached trl-0.24.0-py3-none-any.whl.metadata (11 kB)
Using cached trl-0.24.0-py3-none-any.whl (423 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 99.0 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12
    Uninstalling sympy-1.12:
      Successfully uninstalled sympy-1.12
  Attempting uninstall: trl
    Found existing installation: trl 0.15.2
    Uninstalling trl-0.15.2:
      Successfully uninstalled trl-0.15.2
  U

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.9.0+cu126)
    Python  3.12.9 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers was not installed correctly.
Please install xformers separately first.
Then confirm if it's correctly installed by running:
python -m xformers.info

Longer error message:
xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.9.0+cu126)
    Python  3.12.9 (you have 3.12.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http:/

model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
# --- 1. Define the Test Data ---
test_question = "Explain the difference between a list and a tuple in Python."
test_user_answer = "Lists use square brackets [] and tuples use parentheses (). That is the only difference."

# --- 2. Define the "Judge" Prompt ---
# This is the exact prompt you would use in your website's backend
judge_prompt = f"""You are an expert technical interviewer and Teaching Assistant.
Your task is to evaluate a candidate's answer to a technical interview question.

### The Question:
{test_question}

### The Candidate's Answer:
{test_user_answer}

### Instructions:
1. Analyze the accuracy, completeness, and clarity of the candidate's answer.
2. Provide a score from 0.0 (completely wrong) to 1.0 (perfect).
3. Provide constructive feedback explaining what they got right and what they missed.
4. CRITICAL: You must respond ONLY with a valid JSON object. Do not add markdown formatting.

### Expected Output Format:
{{
    "score": <float>,
    "feedback": "<string>"
}}

### Response:
"""

# --- 3. Run Inference ---
print("Thinking...")
inputs = tokenizer([judge_prompt], return_tensors = "pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens = 512,   # Give it enough room for feedback
    temperature = 0.1,      # Low temp = logic and consistency
    do_sample = True
)

# --- 4. Decode and Print ---
response_text = tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:])[0]
print("\n--- MODEL OUTPUT ---")
print(response_text)

Thinking...

--- MODEL OUTPUT ---
{
    "score": 0.5,
    "feedback": "The candidate correctly identified the difference between lists and tuples. However, they missed the fact that lists are mutable and tuples are immutable. They also did not mention that lists can contain duplicate elements while tuples cannot. Finally, they did not provide any examples to illustrate their points."
}
<|end_of_text|>


In [ ]:
# --- 1. Define the COMPLEX Test Data ---
test_question = "Explain the vanishing gradient problem in Recurrent Neural Networks (RNNs). Why does it happen, and what are two architectural solutions to fix it?"

# This answer has a subtle but major technical error (Gradient Clipping is for Exploding, not Vanishing)
test_user_answer = "The vanishing gradient problem occurs during backpropagation through time when the gradients become extremely small, effectively preventing the weights from updating. This happens because of the chain rule, where we repeatedly multiply derivatives that are smaller than 1. To fix this, you can use Gradient Clipping to cap the gradients, or you can use a smaller learning rate."

# --- 2. Define the Judge Prompt ---
judge_prompt = f"""You are an expert technical interviewer and Teaching Assistant.
Your task is to evaluate a candidate's answer to a technical interview question.

### The Question:
{test_question}

### The Candidate's Answer:
{test_user_answer}

### Instructions:
1. Analyze the accuracy, completeness, and clarity of the candidate's answer.
2. **CRITICAL:** Check for factual errors. Does the proposed solution actually fix the stated problem?
3. Provide a score from 0.0 (completely wrong) to 1.0 (perfect).
4. Provide constructive feedback explaining exactly what is right and what is wrong.
5. Respond ONLY with a valid JSON object.

### Expected Output Format:
{{
    "score": <float>,
    "feedback": "<string>"
}}

### Response:
"""

# --- 3. Run Inference ---
print("Thinking...")
inputs = tokenizer([judge_prompt], return_tensors = "pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens = 512,
    temperature = 0.1,
    do_sample = True
)

# --- 4. Decode and Print ---
response_text = tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:])[0]
print("\n--- MODEL OUTPUT ---")
print(response_text)

Thinking...

--- MODEL OUTPUT ---
{
    "score": 0.8,
    "feedback": "The candidate's answer is mostly correct, but there are some minor errors. The candidate correctly identifies the vanishing gradient problem and provides a solution, but the solution is not entirely accurate. The candidate should have mentioned that the vanishing gradient problem can also occur in other types of neural networks, not just RNNs. Additionally, the candidate should have provided a more detailed explanation of the chain rule and how it relates to the vanishing gradient problem. Overall, the candidate's answer is good, but there is room for improvement."
}
<|end_of_text|>


In [ ]:
# --- 3. Step B: The "Compare & Judge" Prompt ---
judge_prompt = f"""You are a strict technical interviewer.
Evaluate the Candidate's Answer by comparing it to the Reference Answer.

### The Question:
{test_question}

### Reference Answer (Correct):
{gold_standard}

### Candidate's Answer:
{test_user_answer}

### Instructions:
1. Compare the Candidate's solutions to the Reference solutions.
2. If the Candidate suggests 'Gradient Clipping' for Vanishing Gradients, mark it as WRONG. (Clipping is for Exploding).
3. Provide a score (0.0 to 1.0) and feedback.
4. **CRITICAL:** Respond using ONLY the following JSON format:
   {{ "score": 0.0, "feedback": "explanation here" }}

### Response:
"""

# --- 4. Run the Judge ---
print("Judging...")
inputs = tokenizer([judge_prompt], return_tensors = "pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens = 512,
    temperature = 0.1,
    do_sample = True
)

# Decode (skip the prompt tokens to see just the answer)
response_text = tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:])[0]
print("\n--- NEW MODEL OUTPUT ---")
print(response_text)

Judging...

--- NEW MODEL OUTPUT ---
{ "score": 0.0, "feedback": "The Candidate's answer is incorrect. The vanishing gradient problem occurs when the gradients become too small during backpropagation, making it difficult for the network to learn long-term dependencies. This can lead to poor performance and unstable training. The Candidate's suggestion of using Gradient Clipping to cap the gradients is incorrect, as this is a solution for the exploding gradient problem, not the vanishing gradient problem. The Candidate's suggestion of using a smaller learning rate is also incorrect, as this can lead to slow convergence and poor performance." }<|end_of_text|>


In [ ]:
import json

# --- 1. Define the "Good" Test Data ---
test_question = "What is a Python decorator?"
test_user_answer = "A decorator is a function that takes another function as an argument and extends its behavior without explicitly modifying it. In Python, we use the @ symbol to apply them."

# --- 2. Step A: Generate the Reference (Gold Standard) ---
reference_prompt = f"""You are an expert in Python.
Write a concise, technically perfect answer to the following interview question.

Question: {test_question}

Answer:"""

print("Generating Reference Answer...")
inputs = tokenizer([reference_prompt], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
gold_standard = tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:])[0].strip()
print(f"Reference:\n{gold_standard}\n")


# --- 3. Step B: The Judge Step ---
# --- Updated Judge Prompt (Safer Version) ---
judge_prompt = f"""You are a strict technical interviewer.
Evaluate the Candidate's Answer by comparing it to the Reference Answer.

### Question:
{test_question}

### Reference Answer (Correct):
{gold_standard}

### Candidate's Answer:
{test_user_answer}

### Scoring Rubric:
- **Score 1.0 (Perfect):** The answer is factually correct and captures the core concept.
- **Score 0.5 (Partial):** The answer is partially correct but misses the main point.
- **Score 0.0 (Wrong):** The answer is factually incorrect.

### Instructions:
1. Focus on **factual accuracy** and **core concepts**.
2. **DO NOT penalize for brevity.** A concise, correct answer is a perfect answer.
3. **DO NOT require code examples** unless the Question explicitly asked for them.
4. If the Candidate's answer matches the *meaning* of the Reference, give a high score.
5. **CRITICAL:** Respond using ONLY the following JSON format:
   {{ "score": <score_between_0_and_1>, "feedback": "<your_feedback_here>" }}

### Response:
"""

# --- 4. Run Inference ---
print("Judging...")
inputs = tokenizer([judge_prompt], return_tensors = "pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens = 512,
    temperature = 0.1,
    do_sample = True
)

response_text = tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:])[0]
print("\n--- MODEL OUTPUT ---")
print(response_text)

Generating Reference Answer...
Reference:
A Python decorator is a function that takes another function as an argument and returns a function. It is used to modify the behavior of a function without changing its source code. Decorators can be used to add additional functionality to a function, such as logging, caching, or authentication.

Example:

```python
def decorator(func):
    def wrapper(*args, **kwargs):
        # do something before the function is called
        result = func(*args, **kwargs)
        # do something after the function is called
        return result
    return wrapper

@decorator
def my_function(x):
    return x * 2

print(my_function(3)) # output: 6
```

In this example, `my_function` is decorated by `decorator`. When `my_function` is called, `decorator` is called first, which then calls `my_function`. The result of `my_function` is then returned by `decorator`.

In this case, `decorator` does not change the behavior of `my_function`, but it could be used to a

In [ ]:
import json

# --- 1. Test Data (Same as before) ---
test_question = "What is a Python decorator?"
test_user_answer = "A decorator is a function that takes another function as an argument and extends its behavior without explicitly modifying it. In Python, we use the @ symbol to apply them."
# We reuse the 'gold_standard' variable from the previous step if it's already generated.
# If not, un-comment the generation lines below.
# gold_standard = "A Python decorator is a function..."


# --- 2. Step B: The "Chain of Thought" Judge (FIXED) ---
judge_prompt = f"""You are a strict technical interviewer.

### Question:
{test_question}

### Reference Answer:
{gold_standard}

### Candidate's Answer:
{test_user_answer}

### Instructions:
1. First, analyze the Candidate's Answer. Does it contain the key technical points from the Reference?
2. If the answer is correct and clear, give a high score (0.9 or 1.0).
3. **DO NOT penalize for brevity.** Short, correct answers are perfect.
4. **CRITICAL:** Do not copy the example text below. Write your own analysis.

### Output Format (JSON):
{{
    "analysis": "<WRITE YOUR ANALYSIS HERE. List the key concepts the candidate got right or wrong.>",
    "score": <insert_float_score>,
    "feedback": "<insert_constructive_feedback>"
}}

### Response:
"""

# --- 3. Run Inference ---
print("Judging...")
inputs = tokenizer([judge_prompt], return_tensors = "pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens = 512,
    temperature = 0.2,
    do_sample = True
)

response_text = tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:])[0]
print("\n--- MODEL OUTPUT ---")
print(response_text)

Judging...

--- MODEL OUTPUT ---
{
    "analysis": "The candidate's answer is correct and clear. They understand the concept of decorators in Python and how they can be used to modify the behavior of a function without changing its source code. They also use the @ symbol to apply them.",
    "score": 1.0,
    "feedback": "Great job! Keep up the good work."
}
<|end_of_text|>


In [ ]:
import json

# --- 1. The "Partial" Test Case ---
# This answer is true, but useless because it misses the negative side of overfitting
QUESTION = "Explain the concept of Overfitting in Machine Learning."
USER_ANSWER = "Overfitting happens when the model learns the training data really well. It effectively memorizes the training set and has very low error on it."

# --- 2. Step A: Generate Reference ---
reference_prompt = f"""You are an expert in Machine Learning.
Write a concise, 1-sentence technical definition answering the following question.
Focus on the relationship between training performance and testing performance.

Question: {QUESTION}

Answer:"""

print("Generating Reference...")
inputs = tokenizer([reference_prompt], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
gold_standard = tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:])[0].strip().replace("<|end_of_text|>", "")
print(f"Reference: {gold_standard}\n")

# --- 3. Step B: The "Strict Logic" Judge ---
judge_prompt = f"""You are an objective technical grader.

### Question:
{QUESTION}

### Reference (Truth):
{gold_standard}

### Candidate's Answer:
{USER_ANSWER}

### Evaluation Protocol:
1. **Analyze Training Performance:** Does the candidate mention high performance on training data? (Compare to Reference).
2. **Analyze Testing/Generalization:** Does the candidate mention POOR performance on new/unseen data? (Compare to Reference).
3. **Determine Score:**
   - If the Candidate contradicts the Reference, Score = 0.0.
   - If the Candidate is missing a key half of the definition (e.g. ignores test data), Score = 0.5.
   - If the Candidate matches the Reference fully, Score = 1.0.

### Output Format (JSON):
{{
    "analysis": "Reference mentions [X] and [Y]. Candidate mentions [X] but misses [Y].",
    "score": <float>,
    "feedback": "<short_explanation>"
}}

### Response:
"""

print("Judging...")
inputs = tokenizer([judge_prompt], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 512, temperature = 0.1, do_sample = True)

response_text = tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:])[0]
print("\n--- JUDGE OUTPUT ---")
print(response_text)

Generating Reference...
Reference: Overfitting occurs when a model learns the training data too well, resulting in poor performance on new data. It is caused by models that are too complex or have too many parameters, resulting in memorizing the training data instead of learning the underlying patterns. This leads to a high training accuracy but a low generalization accuracy on new data. To avoid overfitting, regularization techniques such as L1 or L2 regularization, early stopping, or dropout can be used.

Judging...

--- JUDGE OUTPUT ---
{
    "analysis": "Reference mentions high training accuracy and poor generalization accuracy. Candidate mentions high training accuracy but misses poor generalization accuracy.",
    "score": 0.5,
    "feedback": "Candidate mentions high training accuracy but misses poor generalization accuracy."
}
<|end_of_text|>


In [ ]:
import json

# --- 1. Define the Test Case ---
# This answer is factually TRUE, but it misses the most important part (performance on new data).
# We expect: High Technical Accuracy, but Lower Completeness.
TEST_DOMAIN = "Machine Learning"
TEST_QUESTION = "Explain the concept of Overfitting."
TEST_USER_ANSWER = "Overfitting happens when a model learns the training data perfectly. It effectively memorizes the noise and details of the training set, resulting in extremely low error during training."

# --- 2. Step A: Generate Reference (with Domain Injection) ---
reference_prompt = f"""You are an expert in {TEST_DOMAIN}.
Write a concise, technically perfect answer to the following interview question.
Focus on the definition and the 'why'. Do NOT use code examples unless absolutely necessary.

Question: {TEST_QUESTION}

Answer:"""

print(f"🔹 Generating Reference for domain '{TEST_DOMAIN}'...")
inputs = tokenizer([reference_prompt], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 256, temperature = 0.2, use_cache = True)
gold_standard = tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:])[0].strip()

print(f"✅ Reference Generated:\n{gold_standard}\n")
print("-" * 40)

# --- 3. Step B: Multi-Metric Judge ---
judge_prompt = f"""You are a strict technical interviewer.

### Question:
{TEST_QUESTION}

### Reference Answer (Truth):
{gold_standard}

### Candidate's Answer:
{TEST_USER_ANSWER}

### Evaluation Protocol:
1. **Analyze:** Compare the Candidate's answer to the Reference. Note matches and misses.
2. **Score Technical Accuracy (0.0-1.0):** Is the information factually correct? (No lies/hallucinations).
3. **Score Completeness (0.0-1.0):** Did they cover the main points? (e.g. missed "test data" in overfitting).
4. **Score Clarity (0.0-1.0):** Is the answer easy to understand?
5. **Overall Score (0.0-1.0):** A weighted average of the above.

### Instructions:
- Be objective.
- **CRITICAL:** Respond using ONLY valid JSON. Do not write anything else.

### Output Format (JSON):
{{
    "analysis": "<Short comparison of Reference vs Candidate>",
    "technical_accuracy": <float>,
    "completeness": <float>,
    "clarity": <float>,
    "overall_score": <float>,
    "feedback": "<Constructive feedback for the student>"
}}

### Response:
"""

print("🔹 Running Judge...")
inputs = tokenizer([judge_prompt], return_tensors = "pt").to("cuda")

# Low temperature (0.1) is crucial for valid JSON and logic
outputs = model.generate(
    **inputs,
    max_new_tokens = 512,
    temperature = 0.1,
    do_sample = True
)

response_text = tokenizer.batch_decode(outputs[:, inputs.input_ids.shape[1]:])[0]

# --- 4. Parse and Display Results ---
print("\n--- 📊 FINAL JSON OUTPUT ---")
try:
    # Clean up potential markdown code blocks
    clean_json = response_text.replace("```json", "").replace("```", "").replace("<|end_of_text|>", "").strip()
    data = json.loads(clean_json)

    print(json.dumps(data, indent=4))

    print("\n--- 🧐 HUMAN CHECK ---")
    print(f"Technical Accuracy: {data.get('technical_accuracy')} (Should be high)")
    print(f"Completeness:       {data.get('completeness')} (Should be medium/low)")
except json.JSONDecodeError:
    print("❌ Failed to parse JSON. Raw output:")
    print(response_text)

🔹 Generating Reference for domain 'Machine Learning'...
✅ Reference Generated:
Overfitting is a phenomenon that occurs when a machine learning model is trained on a dataset that is too small or too noisy, resulting in a model that performs poorly on new, unseen data. This is because the model has learned the specific patterns and noise in the training data, rather than the underlying structure of the data. Overfitting can be avoided by using regularization techniques, such as L1 or L2 regularization, or by using techniques such as cross-validation to ensure that the model is not overfitting the training data.

Overfitting is a common problem in machine learning, and it can be difficult to detect and correct. It is important to understand the concept of overfitting and how to avoid it in order to build accurate and reliable machine learning models.

Here is an example of overfitting in action:

Imagine you have a dataset of images of cats and dogs, and you want to train a machine learni